# Tribble Saldo

calculate workings hours saldo.

In [ ]:
year_span="2022-2023"
country="germany"
territory="B"

In [ ]:
import os
from dateutil.easter import easter

reference_functions = {
    "easter": easter
}

year_span_list = year_span.split('-')

year_span_begin = int(year_span_list[0])
year_span_end = int(year_span_list[1])

APPLICATION_PATH = os.getenv('PURRING_TRIBBLE_HOME_APPLICATION')

## get holidays

In [ ]:
import json
from datetime import datetime, timedelta
import pytz

def check_holiday_in_current_country(country, territory, holiday_dict):
    for country_dict in holiday_dict['countries']:
        if country == country_dict['name'] and territory in country_dict['territories']:
            return True
    return False

with open(f"{APPLICATION_PATH}/public_holidays.json", 'r') as f:
  public_holidays_dict = json.load(f)

holidays = {}

for current_year in range(year_span_begin, year_span_end+1):
    _holidays = []
    for holiday_dict in public_holidays_dict:
        if holiday_dict['type'] == 'fix' and check_holiday_in_current_country(country, territory, holiday_dict):
            day = datetime.strptime(f"{holiday_dict['day']}.{current_year}", "%d.%m.%Y").replace(tzinfo=pytz.utc).date()
        elif holiday_dict['type'] == 'calc' and check_holiday_in_current_country(country, territory, holiday_dict):
            day = reference_functions[holiday_dict['formula']['reference']](current_year) + timedelta(days=holiday_dict['formula']['difference'])
        _holidays.append(day)
    holidays[current_year] = _holidays

## get worktime hours

In [ ]:
from datetime import datetime
import pytz
import sqlite3
import os

tribble_db_fqn = f"{os.getenv('PURRING_TRIBBLE_DATABASE')}"

con = sqlite3.connect(tribble_db_fqn)

cur = con.cursor()

worktime_hours = {}

for current_year in range(year_span_begin, year_span_end+1):
    year_begin_local = pytz.timezone('Europe/Berlin').localize(datetime.strptime(f"01.01.{current_year}", "%d.%m.%Y"))
    year_end_local = pytz.timezone('Europe/Berlin').localize(datetime.strptime(f"01.01.{current_year+1}", "%d.%m.%Y"))

    year_begin_utc = year_begin_local.astimezone(pytz.utc)
    year_end_utc = year_end_local.astimezone(pytz.utc)

    res = cur.execute(f"SELECT timeslot_begin, timeslot_end, hours_distribution FROM worktime_hours w WHERE DATETIME(timeslot_begin) >= DATETIME('{year_begin_utc.strftime('%Y-%m-%d %H:%M:%S')}') AND DATETIME(timeslot_begin) < DATETIME('{year_end_utc.strftime('%Y-%m-%d %H:%M:%S')}');")
    results_list = res.fetchall()

    worktime_hours[current_year] = results_list

con.close()

## get special days

In [ ]:
from datetime import datetime
import pytz
import sqlite3
import os

tribble_db_fqn = f"{os.getenv('PURRING_TRIBBLE_DATABASE')}"

con = sqlite3.connect(tribble_db_fqn)

cur = con.cursor()

worktime_special = {}

for current_year in range(year_span_begin, year_span_end+1):
    year_begin_local = pytz.timezone('Europe/Berlin').localize(datetime.strptime(f"01.01.{current_year}", "%d.%m.%Y"))
    year_end_local = pytz.timezone('Europe/Berlin').localize(datetime.strptime(f"01.01.{current_year+1}", "%d.%m.%Y"))

    year_begin_utc = year_begin_local.astimezone(pytz.utc)
    year_end_utc = year_end_local.astimezone(pytz.utc)

    res = cur.execute(f"SELECT timeslot_begin, timeslot_end, worktime_multiplicator, type FROM worktime_special w WHERE DATETIME(timeslot_begin) >= DATETIME('{year_begin_utc.strftime('%Y-%m-%d %H:%M:%S')}') AND DATETIME(timeslot_begin) < DATETIME('{year_end_utc.strftime('%Y-%m-%d %H:%M:%S')}');")
    results_list = res.fetchall()

    worktime_special[current_year] = results_list

con.close()